In [1]:
import sys
sys.path.append("..")

import pandas as pd

from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.tensorboard import SummaryWriter
from torch_geometric.data import Data
import torch_geometric.nn as nnn
from torch_geometric.utils.convert import from_networkx

from dataset import *
from features import *
from train import *
from model import *

In [2]:
de_df = pd.read_parquet("../data/de_train.parquet")

expressions = torch.tensor(de_df.drop(["cell_type", "sm_name", "sm_lincs_id", "SMILES", "control"], axis=1).values)

In [3]:
import pandas as pd

fi_df = pd.read_csv(r"C:\Users\ghanb\Downloads\Compressed\FIsInGene_070323_with_annotations.txt\FIsInGene_070323_with_annotations.txt", 
                 delimiter="\t")

In [4]:
len(set(fi_df.Gene1.to_list() + fi_df.Gene2.to_list()))

14034

In [9]:
from tqdm import tqdm
import networkx as nx

# gene_list = list(set(fi_df.Gene1.to_list() + fi_df.Gene2.to_list()))

G = nx.Graph()
for gene1, gene2 in tqdm(zip(fi_df["Gene1"].to_list(), fi_df["Gene2"].to_list())):
    G.add_edge(gene1, gene2)
    G.add_edge(gene2, gene1)
G_large = G.subgraph(sorted(nx.connected_components(G))[0])
gene_list = list(G_large.nodes)

278974it [00:00, 475736.21it/s]


In [11]:
with open("../data/temp/c2.cp.reactome.v2023.2.Hs.symbols.gmt") as gmt:
    gmt_data = gmt.readlines()

gene_sets = {}
for l in gmt_data:
    feat_name = l.split("\t")[0]
    gene_sets[feat_name] = l.split("\t")[2:]
    gene_sets[feat_name][-1] = gene_sets[feat_name][-1].split("\n")[0]

data_array = np.zeros((len(gene_list), len(gene_sets)), dtype=np.int16)
for i, (sett, genes) in enumerate(gene_sets.items()):
    idx = []
    for gene in genes:
        if gene in gene_list: idx.append(gene_list.index(gene))
    data_array[idx, i] = 1

In [13]:
pyg_graph = from_networkx(G_large)

In [14]:
pyg_graph.x = data_array

[{'edge_index': tensor([[    0,     0,     0,  ..., 13519, 13520, 13521],
         [    1,     2,     3,  ..., 10889, 13252,  5672]]), 'num_nodes': 13522, 'x': array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=int16)}]

In [13]:
from_networkx(G_large)

Data(edge_index=[2, 557182], num_nodes=13522)

In [36]:
sorted(gene_list)

['A1CF',
 'A2M',
 'A4GNT',
 'AAAS',
 'AADAT',
 'AAG1',
 'AAK1',
 'AAMP',
 'AAR2',
 'AARS1',
 'AARS2',
 'AARSD1',
 'AASDHPPT',
 'AATF',
 'ABAT',
 'ABCA1',
 'ABCA10',
 'ABCA12',
 'ABCA13',
 'ABCA2',
 'ABCA3',
 'ABCA4',
 'ABCA5',
 'ABCA6',
 'ABCA7',
 'ABCA8',
 'ABCA9',
 'ABCB1',
 'ABCB10',
 'ABCB11',
 'ABCB4',
 'ABCB7',
 'ABCB8',
 'ABCB9',
 'ABCC1',
 'ABCC10',
 'ABCC11',
 'ABCC12',
 'ABCC2',
 'ABCC3',
 'ABCC4',
 'ABCC6',
 'ABCC8',
 'ABCC9',
 'ABCD1',
 'ABCD2',
 'ABCD3',
 'ABCD4',
 'ABCE1',
 'ABCF1',
 'ABCF2',
 'ABCG1',
 'ABCG4',
 'ABCG5',
 'ABCG8',
 'ABHD17A',
 'ABHD17B',
 'ABHD17C',
 'ABHD5',
 'ABI1',
 'ABI2',
 'ABL1',
 'ABL2',
 'ABLIM1',
 'ABLIM2',
 'ABLIM3',
 'ABR',
 'ABRAXAS1',
 'ABRAXAS2',
 'ABT1',
 'ACAA1',
 'ACAA2',
 'ACACA',
 'ACACB',
 'ACAD11',
 'ACAD9',
 'ACADL',
 'ACADM',
 'ACADS',
 'ACADVL',
 'ACAN',
 'ACAP1',
 'ACAP2',
 'ACAT1',
 'ACAT2',
 'ACBD3',
 'ACBD4',
 'ACBD5',
 'ACD',
 'ACE',
 'ACE2',
 'ACHE',
 'ACIN1',
 'ACKR1',
 'ACKR2',
 'ACKR3',
 'ACKR4',
 'ACLY',
 'ACO1',
 'ACO2'

In [37]:
with open("../data/temp/c2.cp.reactome.v2023.2.Hs.symbols.gmt") as gmt:
    gmt_data = gmt.readlines()

gene_sets = {}
for l in gmt_data:
    feat_name = l.split("\t")[0]
    gene_sets[feat_name] = l.split("\t")[2:]
    gene_sets[feat_name][-1] = gene_sets[feat_name][-1].split("\n")[0]

data_array = np.zeros((len(gene_list), len(gene_sets)), dtype=np.int16)
for i, (sett, genes) in enumerate(gene_sets.items()):
    idx = []
    for gene in genes:
        if gene in gene_list: idx.append(gene_list.index(gene))
    data_array[idx, i] = 1

In [53]:
data_array[8, :].sum()

8